In [1]:
%load_ext sql

```sql
SELECT * FROM mydatacopy WHERE standard is Null;
SELECT * FROM mydatacopy WHERE standard = Null; --compare this with above query
CREATE TABLE mydata4 AS SELECT * FROM mydatacopy3;
SELECT language, year FROM mydata4 ORDER BY year;
SELECT language, author FROM mydata4 ORDER BY 2 DESC; -- 1 means language, 2 means author
SELECT language, standard FROM mydata4 WHERE standard = 'ANSI';
SELECT language, author FROM mydata4 WHERE YEAR > 1970 ORDER BY language;
SELECT language, year, standard FROM mydata4 WHERE YEAR > 1970 AND standard is Null;
SELECT language, author FROM mydata4 WHERE year BETWEEN 1980 AND 1990;
SELECT language, author FROM mydata4 WHERE year NOT BETWEEN 1980 AND 1990;
INSERT INTO mydata4 VALUES (4, 'Tcl', 'Ousterhout', '1988', NULL); --inserting NULL
INSERT INTO mydata SELECT language, standard FROM mydata4 WHERE standard IS NOT NULL;
UPDATE mydata4 SET year = 1972, standard = 'ANSI' WHERE language  = 'Forth';
DELETE FROM mydata4 WHERE language = 'Forth'; --W/O WHERE clause DELETE will clear all rows

Counting Records -

SELECT COUNT(*) FROM mydata4; --counting records
SELECT COUNT(standard) FROM mydata4;
SELECT COUNT (DISTINCT YEAR) FROM mydata4;
SELECT id, language, author creator FROM mydata4; -- column aliases
SELECT author, language FROM mydata4 WHERE language LIKE 'p%'; --0 or multiple character, case sensitive
SELECT author, language FROM mydata4 WHERE language LIKE '_P_' -- one character per '_'
SELECT language, year - (year%10) decade FROM mydata4;
SELECT language, 'The '||(year/10)*10||'s' decade FROM mydata4; --string operation
SELECT MIN(YEAR) FROM mydata4;
SELECT language, MAX(year) FROM mydata4;  --wrong query
SELECT language, year FROM mydata4 WHERE year = (SELECT MAX(year) FROM mydata4); --correct query
SELECT language, standard FROM mydata4 WHERE standard IS NOT NULL GROUP BY standard, language;
--You cannot group by a column which is not present in the SELECT list.
SELECT name, SUM(salary) FROM employee GROUP BY name; 
SELECT name, SUM(salary) FROM employee WHERE name != 'Dhanraj' GROUP BY name HAVING SUM(salary) < 5000;

joining

SELECT author, language FROM auth, lang WHERE language_id = id;
SELECT author, language FROM auth JOIN lang ON language_id = id;
SELECT author, language FROM auth JOIN lang ON auth.language_id = lang.id; --resolving ambiguity
SELECT l1.language, l2.language AS influenced FROM inflang l1, inflang l2 WHERE l1.id = l2.influenced_by; --self joining
SELECT author FROM auth WHERE language_id IN (SELECT id FROM lang WHERE language = 'TCL'); -- subquery
SELECT author, language FROM auth a,(SELECT id, language FROM lang WHERE year > 1980) n WHERE a.language_id = n.id;
INSERT INTO auth (author_id, author, language_id) VALUES(7, 'Wirth', (SELECT id FROM lang WHERE language = 'Pascal'));
SELECT language, standard FROM lang WHERE standard  = 'ISO' OR standard IS Null;
SELECT year FROM lang WHERE EXISTS(SELECT author FROM auth WHERE language_id = lang.id AND language_id > 4);
CREATE VIEW test AS SELECT language, author  FROM mydata4;  --view
DROP TABLE mydata4; -- error raised because view 'test' depends on this table
DROP TABLE mydata4 CASCADE;

SELECT current_date;
SELECT (5+4)/2; --4
SELECT (5+4)/2.; --4.5
SELECT CHAR_LENGTH('ABCDE');  -- CHARACTER_LENGTH can also be used

SELECT city, temp_lo, temp_hi, prcp, date, location FROM weather, cities WHERE city = name;
SELECT * FROM weather INNER JOIN cities ON (weather.city = cities.name); --alternate form, same as above
SELECT *
FROM weather LEFT OUTER JOIN cities ON (weather.city = cities.name); --columns from left table appear at least once
SELECT * ROM weather RIGHT OUTER JOIN cities ON (weather.city = cities.name); 
SELECT *
FROM weather FULL OUTER JOIN cities ON (weather.city = cities.name); --columns from both table appear at least once

SELECT W1.city, W1.temp_lo AS low, W1.temp_hi AS high,
W2.city, W2.temp_lo AS low, W2.temp_hi AS high
FROM weather W1, weather W2
WHERE W1.temp_lo < W2.temp_lo
AND W1.temp_hi > W2.temp_hi; --self join

ELECT max(temp_lo) FROM weather;
SELECT city FROM weather WHERE temp_lo = max(temp_lo); -- wrong
SELECT city FROM weather WHERE temp_lo = (SELECT max(temp_lo) FROM weather); -- correct
SELECT city, max(temp_lo) FROM weather GROUP BY city;
SELECT city, max(temp_lo) FROM weather GROUP BY city HAVING max(temp_lo) <40;
SELECT city, max(temp_lo) FROM weather WHERE city LIKE 'S%' GROUP BY city HAVING max(temp_lo) <40;

Foreign key -

CREATE TABLE cities1(
city VARCHAR(80) PRIMARY KEY,
location point); 

CREATE TABLE weather1(
city VARCHAR(80) REFERENCES cities1(city),
temp_lo int,
temp_hi int,  --high temperature
prcp real,    --precipitation
date date);

INSERT INTO weather1 VALUES ('Berkeley', 45, 53, 0.0, '1994-11-28'); --error, no corresponding entry in cities1

Let us create 2 tables as shown below -

CREATE TABLE so_headers (
id SERIAL PRIMARY KEY,
customer_id INTEGER,
ship_to VARCHAR (255)
);

CREATE TABLE so_items (
item_id INTEGER NOT NULL, 
so_id INTEGER REFERENCES so_headers(id), --foreign key constraint
product_id INTEGER,
qty INTEGER,
net_price numeric,
PRIMARY KEY (item_id,so_id)
);

foreign key in so_items can also be defined as shown below -

CREATE TABLE so_items (
 item_id INTEGER NOT NULL,
 so_id INTEGER,
 product_id INTEGER,
 qty INTEGER,
 net_price NUMERIC,
 PRIMARY KEY(item_id, so_id),
 FOREIGN KEY(so_id) REFERENCES so_headers(id)
);

Transactions

BEGIN;
UPDATE accounts SET balance = balance - 100.00
WHERE name = 'Alice';
-- etc etc
COMMIT;

Window Function

SELECT depname, empno, salary, avg(salary) OVER (PARTITION BY depname) FROM empsalary;
SELECT depname, empno, salary, rank() OVER (PARTITION BY depname ORDER BY salary DESC) FROM empsalary;
SELECT salary, sum(salary) OVER () FROM empsalary;
SELECT salary, sum(salary) OVER (ORDER BY salary) FROM empsalary;

Inheritance

CREATE TABLE city (
name text,
population real,
altitude int -- (in ft)
);

CREATE TABLE capitals (
state char(2)
) INHERITS (city); -- Notice INHERITS

INSERT INTO city VALUES ('Lucknow', 1000000, 100);
INSERT INTO capitals VALUES ('Lucknow', 1000000, 100,'UP');


ALTER TABLE mydata ADD COLUMN year INTEGER;  --adding column
ALTER TABLE products DROP COLUMN description; -- dropping column
ALTER TABLE mydata RENAME TO language; --renaming table
ALTER TABLE language RENAME COLUMN year TO decade; --renaming column
ALTER TABLE language ALTER COLUMN decade SET DEFAULT 1900; --adding default, only future rows affected
ALTER TABLE language ALTER COLUMN standard SET NOT NULL; --adding constraint
ALTER TABLE products DROP COLUMN description CASCADE; --drop everything which depends on dropped column
ALTER TABLE products ALTER COLUMN price DROP DEFAULT; --removing default value
ALTER TABLE products DROP CONSTRAINT some_name; --dropping constraint
ALTER TABLE products ALTER COLUMN price TYPE numeric(10,2); --convert a column to different type


SELECT * FROM cd.facilities WHERE membercost > 0;
SELECT facid, name, membercost, monthlymaintenance FROM cd.facilities WHERE membercost > 0 and membercost < .02*monthlymaintenance;
SELECT * FROM cd.facilities WHERE name LIKE '%Tennis%';
SELECT membercost, count(*) FROM cd.facilities GROUP BY membercost HAVING count(*) > 1;
SELECT * FROM cd.facilities WHERE facid in (0,5);
SELECT name, CASE WHEN monthlymaintenance > 100 THEN 'expensive' ELSE 'cheap' END AS cost FROM cd.facilities;
SELECT memid, surname, firstname, joindate FROM cd.members WHERE joindate >= '2012-09-01';
SELECT DISTINCT surname FROM cd.members ORDER BY surname LIMIT 10;
SELECT surname FROM cd.members UNION SELECT name  FROM cd.facilities;  -- UNION ALL if duplicates also needed\
SELECT max(joindate) AS latest FROM cd.members;
SELECT firstname, surname, joindate FROM cd.members WHERE joindate = (SELECT max(joindate) FROM cd.members);
SELECT firstname, surname, joindate FROM cd.members ORDER BY joindate DESC LIMIT 1; --last signed up member
SELECT starttime FROM cd.bookings bks INNER JOIN cd.members mems ON bks.memid = mems.memid 
WHERE mems.firstname = 'David' and mems.surname = 'Farrell' ;

SELECT starttime FROM cd.bookings bks ,cd.members mems WHERE mems.firstname = 'David' and mems.surname = 'Farrell'
and mems.memid = bks.memid; --same as above query

SELECT * FROM cd.bookings bks INNER JOIN cd.members mems ON bks.memid = mems.memid;
SELECT bks.starttime, fac.name FROM cd.bookings bks, cd.facilities fac 
WHERE bks.starttime >= '2012-09-21'
AND bks.starttime < '2012-09-22'
AND fac.name LIKE 'Tennis%'
AND bks.facid = fac.facid
ORDER BY bks.starttime;

alternate version of above query
SELECT bks.starttime AS START, fac.name AS name FROM cd.facilities facs INNER JOIN cd.bookings bks ON facs.facid = bks.facid
WHERE facs.facid IN (0,1) AND bks.starttime >= '2012-09-21' AND bks.starttime < '2012-09-21' ORDER BY bks.starttime;

SELECT DISTINCT mem1.firstname, mem1.surname FROM cd.members mem1 INNER JOIN cd.members mem2 
ON mem1.memid = mem2.recommendedby 
ORDER BY mem1.surname, mem1.firstname; --find members who recommended others. No duplicates. Order by (surname, firstname)

/* How can you output a list of all members, including the individual who recommended them (if any)? Ensure that results 
are ordered by (surname, firstname)
*/
SELECT mem1.firstname AS name, mem1.surname AS surname, mem2.firstname AS rec_name, mem2.surname AS rec_surname
FROM cd.members mem1 LEFT OUTER JOIN cd.members mem2 
ON mem2.memid = mem1.recommendedby
ORDER BY surname, name;

/*Produce a list of all members who have used a tennis court? Output should have name of the court, and the name of member 
formatted as a single column. No duplicate data, order by member name
*/

SELECT DISTINCT mems.firstname || ' ' || mems.surname AS member, facs.name AS facility
FROM cd.members mems INNER JOIN cd.bookings bks ON mems.memid = bks.memid
INNER JOIN cd.facilities facs ON bks.facid = facs.facid 
WHERE bks.facid IN (0,1)
ORDER BY member;

/*Produce a list of bookings on the day of '2012-09-14' which will cost the member (or guest) more than 30 USD? Remember that
guests have different costs to members (the listed costs are per half hour slot), and the guest user is always ID 0. Include
in your output the name of facility, the name of member formatted as a single column, and the cost. Order by descending cost,
and don't use subqueries 
*/

SELECT mems.firstname || ' ' || mems.surname AS member, facs.name AS facility,
CASE 
WHEN mems.memid = 0 THEN bks.slots*facs.guestcost
ELSE bks.slots*facs.membercost
END AS cost
FROM cd.members mems INNER JOIN cd.bookings bks ON mems.memid = bks.memid
INNER JOIN cd.facilities facs ON bks.facid = facs.facid
WHERE 
bks.starttime >= '2012-09-14' AND
bks.starttime < '2012-09-15' AND (
(mems.memid = 0 AND bks.slots*facs.guestcost > 30) OR
(mems.memid != 0 AND bks.slots*facs.membercost > 30) )
ORDER BY cost DESC;

-- Part 2- 7th exercise - find out list of members and their recommenders without using joins.

SELECT DISTINCT mems.firstname || ' ' || mems.surname AS member,
(SELECT recs.firstname || ' ' || recs.surname AS recommender 
FROM cd.members recs WHERE recs.memid = mems.recommendedby )
FROM cd.members mems
ORDER by member;

INSERT INTO cd.facilities SELECT 10, 'Spa 2', 20,30, 100000, 800
UNION ALL SELECT 11, 'Squash Court 2', 3.5, 17.5, 5000, 80; --another way to insert data into a table

INSERT INTO cd.facilities SELECT (SELECT max(facid) FROM cd.facilities)+1, 'Spa 3', 20,30,100000, 800; -- another way to insert data

UPDATE cd.facilities SET initialoutlay = 10000 WHERE name = 'Tennis Court 2';
UPDATE cd.facilities SET membercost =  6, guestcost = 30 WHERE facid in (0,1);
UPDATE cd.facilities SET membercost = (SELECT membercost FROM cd.facilities WHERE facid = 0)*1.1, 
guestcost = (SELECT guestcost FROM cd.facilities WHERE facid = 0)*1.1 WHERE facid = 1;
SELECT * FROM cd.facilities;

--6 more elegant solution
UPDATE cd.facilities facs 
SET membercost = facs2.membercost*1.1,
guestcost = facs2.guestcost*1.1
FROM (SELECT * FROM cd.facilities WHERE facid = 0) facs2
WHERE facs.facid = 1;

--To count each facility, you may wish to try out something like this -

SELECT facid, COUNT(*) FROM cd.facilities;


SELECT facid, (SELECT COUNT(*) FROM cd.facilities) FROM cd.facilities; --seems wrong, have to check

--3 produce a count of the number of recommendations each member has made. Order by member

SELECT recommendedby, COUNT(recommendedby) FROM cd.members WHERE recommendedby IS NOT NULL 
GROUP BY recommendedby ORDER BY recommendedby;

--6 find the list of the total no of slots booked per facility per month in the year 2012. Sorted by facid and month

SELECT facid, EXTRACT(MONTH FROM starttime) AS month, SUM(slots) AS "Total Slots" FROM cd.bookings -- ' ' does not work
WHERE starttime >= '2012-01-01' AND starttime <'2013-01-01'
GROUP BY facid, month ORDER BY facid, month;

--7 find the count of members who have made at least one booking
SELECT COUNT(*) FROM (SELECT DISTINCT memid FROM cd.bookings) AS mems -- AS mems part is necessary. Why?

-- Produce a timestamp for 1 a.m. on the 31st of August 2012

SELECT TIMESTAMP '2012-08-31 01:00:00';

--alternate way
SELECT CAST('2012-08-31 01:00:00' AS TIMESTAMP);

--2 find result of subtracting the timestamp '2012-07-30 01:00:00' from timestamp '2012-08-31 01:00:00'

SELECT TIMESTAMP '2012-08-31 01:00:00' - TIMESTAMP '2012-07-30 01:00:00' AS INTERVAL;

---3 generate a list of all the dates in October 2012

SELECT GENERATE_SERIES(TIMESTAMP '2012-10-01', TIMESTAMP '2012-10-31', INTERVAL '1 DAY'); #Postgres specific solution

--4 Get the day of the month from timestamp '2012-08-31'

SELECT EXTRACT (DAY FROM TIMESTAMP '2012-08-31');


```